<a href="https://colab.research.google.com/github/mborkhat/MTech_AI/blob/main/CW/20230820_Lab_5_MTech_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/root/.kaggle/'
!kaggle competitions download -c tensorflow-speech-recognition-challenge
!unzip *.zip
!7za x 'train.7z'

tensorflow-speech-recognition-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  tensorflow-speech-recognition-challenge.zip
replace link_to_gcp_credits_form.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1121103842 bytes (1070 MiB)

Extracting archive: train.7z
--
Path = train.7z
Type = 7z
Physical Size = 1121103842
Headers Size = 389133
Method = Delta LZMA2:24
Solid = +
Blocks = 2

  0%    
Would you like to replace the existing file:
  Path:     ./train/LICENSE
  Size:     18651 bytes (19 KiB)
  Modified: 2017-11-13 21:13:13
with the file from archive:
  Path:     train/LICENSE
  Size:     18651 bytes (19 KiB)
  Modified: 2017-11-13 21:13:13
? (Y)es 

In [ ]:
import librosa
import librosa.display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import IPython.display as ipd
import plotly as py
import plotly.graph_objs as go

In [ ]:
files = []
target_series = []
for dirname, _, filenames in os.walk('/content/train/audio/'):
    for filename in filenames:
        filepath = os.path.join(dirname, filename)
        target = filepath.split('/')[-2]
        if target not in '_background_noise_':
            target_series.append(target)
            files.append(filepath)
data = pd.DataFrame(target_series, columns=['target'])
data['filename'] = files
data = data.sample(frac=1)
data = data.reset_index(drop=True)
data.head()

In [ ]:
data.head()

In [ ]:
df = pd.DataFrame()
for target in data['target'].unique():
    df = df.append(data[data['target']==target].sample(150))
df = df.sample(frac=1)
df = df.reset_index(drop=True)
df.head()

In [ ]:
df.shape

In [ ]:
def create_mfcc_features(path):
    mfccs = []
    try:
        x , sr = librosa.load(path)
        mfccs = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=128)
        mfccs = np.mean(mfccs.T,axis=0)
    except:
        print('Error reading audio')
    return mfccs

In [ ]:
X_df = pd.DataFrame(df['filename'].apply(lambda x: create_mfcc_features(x)).tolist())

In [ ]:
X_df.head()

# MODELS

In [ ]:
X_df.shape

In [ ]:
encoder = LabelEncoder()
encoder.fit(df['target'])
y = encoder.transform(df['target'])

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(np.array(X_df), y, test_size=0.10, stratify=y)

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 16, 8, 1)
x_val = x_val.reshape(x_val.shape[0], 16, 8, 1)

In [ ]:
x_train.shape

In [ ]:
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(16, 8, 1)),
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding = "same"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3), activation='relu', padding = "same"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(len(data['target'].unique()), activation='softmax')
])
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=200)

In [ ]:
prob_val = model.predict(x_val)
pred_val = np.argmax(prob_val, axis=1)
pred_val = encoder.inverse_transform(pred_val)

In [ ]:
sns.heatmap(confusion_matrix(encoder.inverse_transform(y_val), pred_val))

In [ ]:
# x , sr = librosa.load('/content/my_audio.mp4')
# plt.figure(figsize=(20, 5))
# plt.title('Waveplot')
# librosa.display.waveplot(x, sr=sr)
# plt.show()

In [ ]:
x.shape

In [ ]:
total_time = x.shape[0]/sr
per_window = x.shape[0]/total_time
total_time, per_window

In [ ]:
ipd.Audio('/content/my_audio.mp4')

In [ ]:
x , sr = librosa.load('/content/my_audio.mp4')
plt.figure(figsize=(20, 5))
plt.title('Waveplot')
librosa.display.waveplot(x[int(2.3*per_window):int(2.6*per_window)], sr=sr)
plt.show()

In [ ]:
mfccs = librosa.feature.mfcc(y=x[int(2.3*per_window):int(2.6*per_window)], sr=sr, n_mfcc=128)
mfccs = np.mean(mfccs.T,axis=0)

In [ ]:
mfccs.shape

In [ ]:
x_test = mfccs.reshape(1, 16, 8, 1)
x_test

In [ ]:
prob_val = model.predict(x_test)
pred_val = np.argmax(prob_val, axis=1)
pred_val = encoder.inverse_transform(pred_val)

In [ ]:
pred_val